# M2 Brief 2 : Nettoyage et Préparation Éthique d'un Jeu de Données

## Contexte de l'étude


Ce notebook va être utiliser pour préparer un dataset dans le but d'entrainer un modèle destiné à faire une regression pour déterminé le montant de pret maximal que l'on peux accorder à un emprunteur.

On va procéder en 2 étapes :
* **Analyse technique et nettoyage des données** : Le but étant d'avoir un dataset le plus exploitable possible sans aucunes considération métier ou éthique
* **Adaptation du dataset** : La prise en compte des contraintes métier et réglementaire ainsi que les décisions éthique pour la modification de ce dataset afin qu'il réponde à ces 3 paramètres

Remarques :

## Bootstrap
Ces bloc sont à exécuter pour installer et charger les dépendances

In [ ]:
# Preparation de l'environnement
!pip install -r requirements.txt

In [ ]:
# Chargement des modules
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer


## Chargement des données et affichage des données brutes

In [ ]:
df = pd.read_csv('data/dataset.csv')
# Suppression des doublons des le début
df.drop_duplicates(inplace=True)
print(f"Le dataset comporte {df.shape[0]} lignes et {df.shape[1]} colonnes")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
def calcul_taux_manquant():
    total_rows = df.shape[0]
    # Calcul des pourcentage de lignes manquantes
    historique_credits_rows = df["historique_credits"].count()
    situation_familiale_rows = df["situation_familiale"].count()
    score_credit_rows = df["score_credit"].count()
    loyer_mensuel_rows = df["loyer_mensuel"].count()
    historique_credits_pc_vide = 1 - historique_credits_rows/total_rows
    situation_familiale_pc_vide = 1 - situation_familiale_rows/total_rows
    score_credit_pc_vide = 1 - score_credit_rows/total_rows
    loyer_mensuel_pc_vide = 1 - loyer_mensuel_rows/total_rows
    pc_vide = pd.DataFrame({
        "historique_credits": [historique_credits_pc_vide],
        "situation_familiale": [situation_familiale_pc_vide],
        "score_credit": [score_credit_pc_vide],
        "loyer_mensuel": [loyer_mensuel_pc_vide],
    })
    return pc_vide
print("% données vide au chargements")
calcul_taux_manquant()

On constate que plus de la moitier des données relatives a l'évaluation des crédits sont manquantes, le dataset étant orienté sur l'attribution du montant maximal accordé à un emprunteur ces noté sont d'une importance notable.
Il va falloir détertminer comment on va traiter ces valeurs
Concernant la situation_familiale et le loyer_mensuel il faut vérifier si il y'a correlation avec le montant maximum du pret accorder, en fonction de quoi cette données sera exploité ou non.

On cherche s'il y'a correlation entre les valeurs manquantes

In [ ]:
msno.heatmap(df, figsize=(8, 4))

Il n'y a pas de correlation forte entre les données manquantes

### Premières observations

En première analyse rapide on constate :
* Qu'il y'a un ensemble de colonnes ayant des valeurs manquantes, il faudra déterminer les impactes
* On à des valeurs négatives pour le loyer ce qui est plutôt anormal
* On à un percentile 25% identique au minimum pour le montant du pret, ce qui semble indiquer une forte présence de pret à 500 €
* Le poids semble comporter des valeurs aberrantes surtout dans les valeurs les plus petites

Il faudra confirmer ou infirmer ces tendances.

In [ ]:
# df.drop(['nom', 'prenom', 'date_creation_compte'], axis=1, inplace=True)
df.head()

## Analyse de la distribution des valeurs

On trace les histogramme de chaque features histoire de voir rapidement la distribution des valeurs

In [ ]:
# Determine le nombre de colonne souhaité pour l'affichage du subplot
num_features = df.shape[1] # nombre de features
num_cols = 3 # nombre de colonnes
num_rows = (num_features + num_cols - 1) // num_cols # nombre de lignes

plt.figure(figsize=(num_cols * 5, num_rows * 4)) # On définie la largeur et hauteur des graphe en fonction des valeurs

for i, column in enumerate(df.columns):
    plt.subplot(num_rows, num_cols, i + 1)
    sns.histplot(df[column])
    plt.title(f'Histogramme de {column}')
    plt.xlabel(column)
    plt.ylabel('Fréquence')

plt.tight_layout()

Il apparait que les données sont globalement correctement distribuées, il y'a cependant quelques features qui semble avoir des données étranges pour les revenus_estimes, le montant_pret et le loyer_mensuel.
Le cas de la nationalité est un peu plus complexe car on ne connais pas exactement comment à été construit le dataset, cette valeur necessitera une analyse plus appronfondi en fonction du contexte de l'obtention de celui-ci.

### revenu estime mois

On zoom sur la partie basse du graphe pour trouver la valeur la plus représentée

In [ ]:
sns.histplot(df[df['revenu_estime_mois'] <= 1000]['revenu_estime_mois'])

On constate une trés forte de concentration de revenue à 500 €, on vérifie ca avec des valeurs numériques

In [ ]:
print("Revenus = 500 :", df[df['revenu_estime_mois'] == 500]['revenu_estime_mois'].count())
print("Revenus entre 501 et 1000 :", df[(df['revenu_estime_mois'] > 500) & (df['revenu_estime_mois'] <= 1000)]['revenu_estime_mois'].count())
df[df['revenu_estime_mois'] <= 1000]['revenu_estime_mois'].value_counts(bins=[0.0, 500, 600, 700, 800, 900, 1000], sort=False)

On constate que la majorité des bas revenue sont plutôt autour de 500, on peux considérer que cette valeurs est une valeur.

On va regarder rapidement si il y'a une correlation entre les revenues faible et le montant des pret accordés autour de 500, le cas des montant de pret sera traité plus loins dans ce document.

In [ ]:
df_bas_revenue = df[df['revenu_estime_mois'] <= 1000]
df_bas_revenue[['revenu_estime_mois', 'montant_pret']].corr()


In [ ]:
df[['revenu_estime_mois', 'montant_pret']].corr()

Il ne semble pas y avoir de correlation franche entre les faible revenus et le montant du pret pour ceux ci, contrairement à la situation pour tout le dataset. On ne peux pas établir que les petit revenus donne lieux uniquement à des pret de 500.

### Loyer mensuel

On voit 2 pics sur les loyers mensuels, beaucoup de loyer à 5000 et à 10000

In [ ]:
df[df['loyer_mensuel'] > 4000]['loyer_mensuel'].value_counts()

Une winsorisation des valeurs de loyer n'aurais pas de sens car trop de valeurs hors limites supérieur sont présentes, le 99eme quantil reste à 10000.


In [ ]:
q1 = df['loyer_mensuel'].quantile(0.01)
print(f"Quantile = {q1}")
q99 = df['loyer_mensuel'].quantile(0.99)
print(f"Quantile = {q99}")

On regarde la distribution des valeurs en excluant ce valeurs extremes.

In [ ]:
sns.histplot(df[df['loyer_mensuel'] < 5000]['loyer_mensuel'])

In [ ]:
sns.scatterplot(data=df[df['loyer_mensuel'] < 5000], x='loyer_mensuel', y='revenu_estime_mois', hue='region')

Il n'y a pas de correlation franche entre les revenus estimé et la region avec les loyers mensuel.
On va utiliser les plus proche voisins pour reconstruire les montants des loyers pour les valeurs extremes.

On décide de se baser sur la region et le revenu_estime_mois


In [ ]:
df["region"].unique()

In [ ]:
df['region_num'] = df['region'].map({
    'Occitanie': 0,
    'Île-de-France': 1,
    'Auvergne-Rhône-Alpes': 2,
    'Corse': 3,
    'Bretagne': 4,
    'Hauts-de-France': 5,
    'Provence-Alpes-Côte d’Azur': 6,
    'Normandie': 7
})

In [ ]:
# pd.to_numeric(df[df['loyer_mensuel'] >= 5000]['loyer_mensuel']).unique()
df[df['loyer_mensuel'] >= 5000]['loyer_mensuel'].info()

In [ ]:
# On met à balcn les valeurs extremes
df.loc[df['loyer_mensuel'] >= 5000, 'loyer_mensuel'] = np.nan

imputer = KNNImputer(n_neighbors=5)
df[["revenu_estime_mois", "region_num", "loyer_mensuel"]] = imputer.fit_transform(
    df[["revenu_estime_mois", "region_num", "loyer_mensuel"]]
)

# On supprime la colonne temporaire 'region_num' utilisé juste pour les calcules
df.drop('region_num', axis=1, inplace=True)

In [ ]:
sns.histplot(df['loyer_mensuel'])

Cette fois ci, on va faire une winsorisation

In [ ]:
q1 = df['loyer_mensuel'].quantile(0.01)
print(f"Quantile = {q1}")
q99 = df['loyer_mensuel'].quantile(0.99)
print(f"Quantile = {q99}")
df['loyer_mensuel'] = np.clip(df['loyer_mensuel'], q1, q99)
sns.histplot(df['loyer_mensuel'])

In [ ]:
calcul_taux_manquant()

On à ramener à 0 le nombre de loyen mensuel manquant grace aux estimations

### montant de pret

On constate un grand pic de pret dans les montants inférieurs, on affine l'analyse.

In [ ]:
sns.histplot(df[df['montant_pret'] <= 10000]['montant_pret'])

In [ ]:
sns.histplot(df[df['montant_pret'] <= 1000]['montant_pret'])
plt.show()
df[df['montant_pret'] <= 1000]['montant_pret'].value_counts(bins=[0.0, 500, 600, 700, 800, 900, 1000], sort=False)

On constate une grande présence de pret à 500, ce sont des pret de faible valeurs, on conserve ces valeurs inchangée car on considère que ce sont des prêt standard.

In [ ]:
df_imputer = df.copy()
# On applique l'imputer KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_imputer[["age", "revenu_estime_mois",  "risque_personnel", "montant_pret", "historique_credits"]] = imputer.fit_transform(
    df_imputer[["age", "revenu_estime_mois",  "risque_personnel", "montant_pret", "historique_credits"]]
)
# L'imputer travaillant sur des nombre flottant, on converti la colonne en antier pour rester cohérent dans les données.
df_imputer['historique_credits'] = df_imputer['historique_credits'].astype(int)

sns.histplot(df_imputer['historique_credits'])

On constate une modification significative de la distribution, on va donc ignorer cette méthode, pour la suite du traitement on va supprimer les lignes ou ce score est manquant, mais avant on conserve le dataset en l'état pour le calcul du score_credit.

### Determination de correlation entre historique_credits et score_credit

On va utiliser ici une méthode manuelle à des fins d'apprentissage, voir si on a une relation entre ces 2 valeurs, ca permettra de déterminer comment on pourrait éventuellement reconstruire les valeurs manquantes


In [ ]:
df_histo_score_credit = df[df["historique_credits"].notna() & df["score_credit"].notna()]
print(df_histo_score_credit.shape)
nb_full_ligne = df_histo_score_credit.shape[0]
sns.pointplot(data=df_histo_score_credit, x="historique_credits", y="score_credit", linestyles='none')

### Calcul de l'historique credit manquant

Pour cela on va travailler sur une copie des données et utiliser le KNNImputer pour reconstruire les valeurs manquantes basé sur l'age, le revenu estimé, le risque personnel et le montant du pret.

De cette manière on pourra estimer si il est pertinent d'utiliser cette méthode.

Par rapport aux données existantes il semble y avoir une tendance qui se dessine entre l'historique de crédit et le score crédit. On semble plus faire confiance aux personnes qui n'ont pas de crédit ou qui en ont fait l'exprerience a plusieurs reprises.

On va tenter d'obtenir des valeurs plus précise sur cette éventuelle correlation. Voyons avec des valeurs numériques.

In [ ]:
df_histo_score_credit.groupby("historique_credits")["score_credit"].describe()

Ca semble confirmer ce qui apparaissait visuellement dans le graphe, on peux essayer de recontruire les valeurs manquantes sur ces bases. On va prendre la moyenne comme valeur appliquée.

On ne peut appliquer ces valeurs qu'aux colonnes dont l'historique crédit est définie mais pas le score crédit. On va vérifier si beaucoup de lignes sont dans ce cas

In [ ]:
print(df[df["historique_credits"].notna() & df["score_credit"].isna()].shape)

2520 lignes sont dans ce cas, On va donc faire la substitution


In [ ]:
# On substitue les valeurs vides avec les valeurs calculées
df.loc[(df["historique_credits"] == 0.0) & df["score_credit"].isna(), 'score_credit'] = 580
df.loc[(df["historique_credits"] == 1.0) & df["score_credit"].isna(), 'score_credit'] = 553
df.loc[(df["historique_credits"] == 2.0) & df["score_credit"].isna(), 'score_credit'] = 577
df.loc[(df["historique_credits"] == 3.0) & df["score_credit"].isna(), 'score_credit'] = 581
df.loc[(df["historique_credits"] == 4.0) & df["score_credit"].isna(), 'score_credit'] = 576
df.loc[(df["historique_credits"] == 5.0) & df["score_credit"].isna(), 'score_credit'] = 557

# On vérifie que les valeurs sont correctement substitué
print(df[df["historique_credits"].notna() & df["score_credit"].isna()].shape)

Le résultatindique bien qu'il n'y à plus de lignes où `score_credit` est vide lorsqu'on à une valeur dans `historique_credit`.

On va maintenant comparer les valeurs de substitution avec les données réelles lorsque les deux valeurs sont présentes, très logiquement il ne devrait pas y avoir de gros eccarts dans les moyennes et l'eccart type devrait se réduire car on insère beaucoup de valeurs moyennes.

In [ ]:
df_histo_score_credit_rebuild = df[df["historique_credits"].notna() & df["score_credit"].notna()]
print(df_histo_score_credit_rebuild.shape)
nb_full_ligne_rebuild = df_histo_score_credit_rebuild.shape[0]
print("nombre de ligne reconstruite : ", nb_full_ligne_rebuild - nb_full_ligne)
# Nombre de ligne complétée
sns.pointplot(data=df_histo_score_credit_rebuild, x="historique_credits", y="score_credit", linestyles='none')
plt.show()
df_histo_score_credit_rebuild.groupby("historique_credits")["score_credit"].describe()

Comme prévu les moyennes ont peu bougées et l'eccart type s'est reserré.



#### Conclusion

On va conserver cette approche pour compléter le score crédit pour la suite du traitement.


In [ ]:
calcul_taux_manquant()

On a pu réduire de moitier l'absence de valeur pour le score crédit.

Maintenant on peux supprimer toutes les lignes où historique_credits n'est pas définie.

In [ ]:
df = df.dropna(subset=['historique_credits'])
calcul_taux_manquant()

### Situation familiale

On a vu que la distribution de cette ligne est trés uniforme, on choisis de mettre des valeurs aléatoire parmi celle existante lorsque la valeur n'est pas définie. On pourrait prendre la valeur la plus représenter via un imputer, mais ca risquerait de déséquilibré le dataset.

In [ ]:
np.random.seed(42) # On fixe la seed pseudo aléatoire sur la réponse à la grande question, qu'est ce que la vie, l'univers et tout le reste. Ceci afin de garantir d'avoir une reconstruction identique à chaque éxecution.
type_situation = df[df['situation_familiale'].notna()]['situation_familiale'].unique() # determination de la liste des valeurs possibles
print(type_situation)
taille_isna=df['situation_familiale'].isna().sum() # nombre de ligne vide
print(taille_isna)
tableau_random = np.random.choice(type_situation, taille_isna) # creer un tableau de valeur aléatoire
df.loc[df['situation_familiale'].isna(), 'situation_familiale'] = tableau_random # on bourre les valeur vide avec les valeurs aléatoire récupérées

calcul_taux_manquant()


### Traitement de nottoyage simple des colonnes taille et poids

Les colonnes ayant quelques valeurs aberrantes en faible quantités, on va appliquer la winsorisation à celle-ci pour les remettre dans les valeurs correctes

In [ ]:
for colonne in ['poids', 'taille']:
    q1 = df[colonne].quantile(0.01)
    print(f"Quantile {colonne} = {q1}")
    q99 = df[colonne].quantile(0.99)
    print(f"Quantile {colonne} = {q99}")
    df[colonne] = np.clip(df[colonne], q1, q99)


### Conclusion

On à pu recontruire un dataset complet en faisant différent choix de reconstruction et de suppression de données. On enregistre celui-ci sous le nom `reconstructed_dataset.csv` dans le dossier data.



In [ ]:
df.to_csv('data/00-reconstructed-dataset.csv', index=False, encoding='utf-8-sig')

## Analyse éthique

Maintenant qu'on à un dataset reconstruit, on va faire une étude de corrélation pour voir les critère qui font varier les montant de pret maximaux.
On utilise un ensemble de pairgrid pour avoir un apperçu rapide.

On reviendra sur une étude plus précise si on constate des valeurs un peu complexe à analyser.

In [ ]:
# au lieu d'utiliser map, cette fois ci on utilise factorize.
# df['sexe_num'], labels_uniques = pd.factorize(df['sexe'])
# print(labels_uniques)
# df['sport_licence_num'], labels_uniques = pd.factorize(df['sport_licence'])
# print(labels_uniques)
# df['niveau_etude_num'], labels_uniques = pd.factorize(df['niveau_etude'])


#### On regarde si la situation familiale a une influence.

In [ ]:
sns.pairplot(df, hue='situation_familiale')

Visiblement pas d'influence notable.

**On supprime cette données du dataset**

In [ ]:
# On supprime la colonne
df.drop('situation_familiale', axis=1, inplace=True)

#### Influence du sexe

In [ ]:
sns.pairplot(df, hue='sexe')

Dans le cas du sexe, il semble se dégager un léger favoritisme pour les hommes. Cependant il est éthiquement discutable de favoriser le sexe de l'emprunteur.

**On supprime donc cette valeur du dataset**


In [ ]:
# On supprime la colonne
df.drop('sexe', axis=1, inplace=True)

#### Influence du poids

In [ ]:
sns.pairplot(df, hue='poids')

Il ne semble pas y avoir de correlation entre le poids et le montant du pret, on veux tout de même vérifier via une correlation car pour les assurance lié à la santé c'est un critère pris en compte. On double check la valeur pour éviter une mauvaise interpretation.


In [ ]:
df[['poids', 'montant_pret']].corr()

Le poids n'a pas d'inflkuence sur le montant du pret

**On supprime cette colonne du dataset**

In [ ]:
df.drop('poids', axis=1, inplace=True)

#### Influence de la taille

In [ ]:
sns.pairplot(df, hue='taille')

Là encore pas de correlation établie

**On supprime la colonne**

In [ ]:
df.drop('taille', axis=1, inplace=True)

#### Influence de la pratique sportive

In [ ]:
sns.pairplot(df, hue='sport_licence')

Il semble y avoir une légère correlation entre le montant préter et la pratique sportive. On vérifie via la correlation.

In [ ]:
df['sport_num'] = df['sport_licence'].map({
    'non': 0,
    'oui': 1
})
df[['montant_pret', 'sport_num']].corr()

Il y'a effectivement une légère correlation entre ces 2 facteurs, cependant on choisis de ne pas le prendre en compte car ça concerne la vie privé de l'emprunteur.

**On supprime cette colonne**

In [ ]:
df.drop(['sport_licence', 'sport_num'], axis=1, inplace=True)

#### Influence de l'age

In [ ]:
sns.pairplot(df, hue='age')

Il ne semble pas y avoir de correlation entre l'age et le montant du pret

**On supprime cette colonne**

In [ ]:
df.drop('age', axis=1, inplace=True)

#### Influence de fumeur ou non

In [ ]:
sns.pairplot(df, hue='smoker')

On constate que le status fumeur à une influence sur le montant du pret

**On conserve cette valeur dans le dataset car c'est un facteur de risque important sur le remboursement du pret**

#### Influence du revenu mensuel estime

In [ ]:
sns.pairplot(df, hue='revenu_estime_mois')

Il semble y avoir une correlation entre le montant du pret et le revenu mensuel. On vérifie.

In [ ]:
df[['montant_pret', 'revenu_estime_mois']].corr()

Il y'a bien correlation.

**On concerve cette colonne**

#### Correlation du niveau d'étude

On part du postulat que les revenu estimé sont correlé au niveau d'étude, on vérifie que c'est bien le cas

In [ ]:
sns.scatterplot(x=df['revenu_estime_mois'], y=df['niveau_etude'])

In [ ]:
df['etude_num'] = df['niveau_etude'].map({
    'aucun': 0,
    'bac': 1,
    'bac+2': 2,
    'master': 3,
    'doctorat': 4
})
df[['etude_num', 'revenu_estime_mois']].corr()

Visiblement il n'y a pas de correlation entre les revenus mensuels et le niveau d'étude dans le dataset. On ne peux pas abandonné cette valeur sous pretexte qu'elle est corrélé au revenue mensuel.

On nettoie la colonne numérique

In [ ]:
df.drop('etude_num', axis=1, inplace=True)

In [ ]:
#### relation avec la region

In [ ]:
sns.pairplot(df, hue='region')

Le montant du pret semble influencé par la région, surtout sur les pret de faible montant.

**On conserve la colonne**

#### Correlation de la nationalité

In [ ]:
sns.pairplot(df, hue='nationalité_francaise')

Il y'a une nette correlation entre le montant des pret et la nationalité, les emprunteur non français on majoritairement des pret pas plus élevé que 500.

**On va tout de même choisir d'exclure cette données car elle est descriminatoire.**

In [ ]:
df.drop('nationalité_francaise', axis=1, inplace=True)

#### Correlation entre historique credits et score credit

On regroupe ces deux colonnes car il y'a une correlation entre historique crédit et score credit. On va cependant se contenté de faire la correlation sur le score credit.

In [ ]:
sns.pairplot(df, hue='score_credit')
plt.show()

In [ ]:
df[['historique_credits', 'score_credit', 'montant_pret']].corr()

Il semble ne pas y avoir une forte correlation entre le montant du crédit et le score crédit, cependant on va garder cette valeur car celle-ci semble importante dans le context des crédit.

#### Risque personnel

On cherche la correlation potentiel sur le risque personnel.

In [ ]:
sns.pairplot(df, hue='risque_personnel')

In [ ]:
df[['risque_personnel', 'montant_pret']].corr()

Il n'y a pas de correlation entre ce risque personnel et le montant du pret.

**On supprime cette colonne**

In [ ]:
df.drop('risque_personnel', axis=1, inplace=True)

## Construction du dataset final

On conserve les colonnes : `smoker`, `region`, `niveau_etude`, `revenu_estime_mois`, `historique_credit`, `score_credit`, `loyer_mensuel`, `montant_credit`.

Apres cette analyse et comme les colonnes ont été supprimé au fur et a mesure on sauvegarde le fichier sous le nom `01-final-dataset.csv` qui pourra être utilisé pour l'entrainement d'un modèle de regression. Dans ce cas là les features seront `smoker`, `region`, `niveau_etude`, `revenu_estime_mois`, `historique_credit`, `score_credit`, `loyer_mensuel` et la target `montant_credit`.


In [ ]:
# On ne transforme plus smoker en valeur numérique
# df['smoker'] = df['smoker'].map({
#     'non': 0,
#     'oui': 1
# })

In [ ]:
df.to_csv('data/01-final-dataset.csv', index=False, encoding='utf-8-sig')
# On supprime nom, prenom et date_creation_compte
df.drop(['nom', 'prenom', 'date_creation_compte'], axis=1, inplace=True)
df.to_csv('data/training_dataset.csv', index=False, encoding='utf-8-sig')